<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-pandas" data-toc-modified-id="Introduction-to-pandas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to pandas</a></span><ul class="toc-item"><li><span><a href="#Dataframe-construction-from-numpy-arrays-and-scipy-sparse-arrays" data-toc-modified-id="Dataframe-construction-from-numpy-arrays-and-scipy-sparse-arrays-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataframe construction from numpy arrays and scipy sparse arrays</a></span></li><li><span><a href="#Unseen-categorical-values-in-a-pandas-dataframe" data-toc-modified-id="Unseen-categorical-values-in-a-pandas-dataframe-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Unseen categorical values in a pandas dataframe</a></span></li><li><span><a href="#Defining-categorical-values-with-order" data-toc-modified-id="Defining-categorical-values-with-order-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Defining categorical values with order</a></span></li></ul></li><li><span><a href="#Add-new-columns-saving-Nan-values-get_dummies_nan" data-toc-modified-id="Add-new-columns-saving-Nan-values-get_dummies_nan-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add new columns saving Nan values <code>get_dummies_nan</code></a></span></li><li><span><a href="#Compute-nan-features-for-rows:-nan_features" data-toc-modified-id="Compute-nan-features-for-rows:-nan_features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compute nan features for rows: <code>nan_features</code></a></span></li><li><span><a href="#Substitute-np.NaN-by-other-values:-MissingImputer-class" data-toc-modified-id="Substitute-np.NaN-by-other-values:-MissingImputer-class-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Substitute <code>np.NaN</code> by other values: <code>MissingImputer</code> class</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Improving-MissingImputer" data-toc-modified-id="Improving-MissingImputer-4.0.0.0.1"><span class="toc-item-num">4.0.0.0.1&nbsp;&nbsp;</span>Improving MissingImputer</a></span></li></ul></li></ul></li></ul></li></ul></li><li><span><a href="#New-columns-from-a-df-column-containing-lists-as-items-(byvalue)" data-toc-modified-id="New-columns-from-a-df-column-containing-lists-as-items-(byvalue)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>New columns from a df column containing lists as items (<code>byvalue</code>)</a></span><ul class="toc-item"><li><span><a href="#New-columns-from-a-df-column-containing-lists-as-items-(bykeyvalue)" data-toc-modified-id="New-columns-from-a-df-column-containing-lists-as-items-(bykeyvalue)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>New columns from a df column containing lists as items (<code>bykeyvalue</code>)</a></span></li></ul></li><li><span><a href="#New-columns-from-a-df-column-containing-lists-as-items-(byorder)" data-toc-modified-id="New-columns-from-a-df-column-containing-lists-as-items-(byorder)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>New columns from a df column containing lists as items (<code>byorder</code>)</a></span></li></ul></div>

# Introduction to pandas

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import copy
from collections.abc import Iterable 
import itertools

## Dataframe construction from numpy arrays and scipy sparse arrays

Pandas Dataframes can be constructed from `np.ndarray` objects

In [2]:
X = np.array([[0,0,1],[1,0,0]])
X

array([[0, 0, 1],
       [1, 0, 0]])

In [3]:
df_X = pd.DataFrame(X)
df_X

,0,1,2
0,0,0,1
1,1,0,0


Moreover, we can also construct Dataframes from sparse arrays

In [4]:
X_sp = scipy.sparse.csr_matrix([[0,0,1],[1,0,0]])
X_sp

<2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [5]:
pd.DataFrame.sparse.from_spmatrix(X_sp)

,0,1,2
0,0,0,1
1,1,0,0


## Unseen categorical values in a pandas dataframe

An annoying detail that might happen when doing a train test split is that the splits might have different unique values in a particular column. This will mean that the one hot encoding done by pandas will be different for the different splits. 

Let us see the problem

In [6]:
df_cars = pd.DataFrame([[2,"mercedes","middleclass"], 
                        [np.NaN,"mercedes","middleclass"],
                        [3,"Audi",np.NaN]],
                        columns= ["members","vehicles","status"])

In [7]:
df_cars

,members,vehicles,status
0,2.0,mercedes,middleclass
1,NaN,mercedes,middleclass
2,3.0,Audi,NaN


In [8]:
pd.get_dummies(df_cars)

,members,vehicles_Audi,vehicles_mercedes,status_middleclass
0,2.0,0,1,1
1,NaN,0,1,1
2,3.0,1,0,0


If I do a the one hot encoding in a subset where there are different values the number of columns will be different:

In [9]:
pd.get_dummies(df_cars[0:2])

,members,vehicles_mercedes,status_middleclass
0,2.0,1,1
1,NaN,1,1


In [10]:
df_cars["vehicles"]

0    mercedes
1    mercedes
2        Audi
Name: vehicles, dtype: object

Now we can cast a column with type "category"

In [11]:
from pandas.api.types import CategoricalDtype

cats =  df_cars["vehicles"].unique()
cat_type = CategoricalDtype(categories=cats, ordered=True)

df_cars[["vehicles"]].astype(cat_type)
df_cars[["vehicles"]] = df_cars[["vehicles"]].astype("category")
df_cars["vehicles"]

0    mercedes
1    mercedes
2        Audi
Name: vehicles, dtype: category
Categories (2, object): ['Audi', 'mercedes']

With the type "category" now the one hot encoding will produce a column for each of the allowed values [Audi, mercedes]. This will work even if in the provided dataframe to `get_dummies` some values are not present in a column.

In [12]:
pd.get_dummies(df_cars[0:2])

,members,vehicles_Audi,vehicles_mercedes,status_middleclass
0,2.0,0,1,1
1,NaN,0,1,1


Notice that we can add more values with **`df_cars[col].cat.add_categories([value1, value2])`**.

For example we can add now the category "renault" to our "vehicles" column

In [13]:
df_cars["vehicles"] = df_cars["vehicles"].cat.add_categories(["renault"])

In [14]:
df_cars["vehicles"]

0    mercedes
1    mercedes
2        Audi
Name: vehicles, dtype: category
Categories (3, object): ['Audi', 'mercedes', 'renault']

And if we do the one hot encoding we see that we get 3 columns for "vehicles".

In [15]:
pd.get_dummies(df_cars[0:2])

,members,vehicles_Audi,vehicles_mercedes,vehicles_renault,status_middleclass
0,2.0,0,1,0,1
1,NaN,0,1,0,1


## Defining categorical values with order

We can provide an order to the different categories, when the one hot encoding is done it will do it respecting the order

In [16]:
df_cars = pd.DataFrame([[2,"seat","middleclass"], 
                        [np.NaN,"mercedes","middleclass"],
                        [3,"Audi",np.NaN]],
                        columns= ["members","vehicles","status"])

In [17]:
categories=['seat','bmw','mercedes']
car_type = pd.api.types.CategoricalDtype(categories=categories, ordered=True)
df_cars[["vehicles"]] = df_cars[["vehicles"]].astype(car_type)

In [18]:
df_cars["vehicles"]

0        seat
1    mercedes
2         NaN
Name: vehicles, dtype: category
Categories (3, object): ['seat' < 'bmw' < 'mercedes']

In [19]:
pd.get_dummies(df_cars)

,members,vehicles_seat,vehicles_bmw,vehicles_mercedes,status_middleclass
0,2.0,1,0,0,1
1,NaN,0,0,1,1
2,3.0,0,0,0,0


Moreover, we can do comparisons using the specified order

In [20]:
df_cars["vehicles"] < "mercedes"

0     True
1    False
2    False
Name: vehicles, dtype: bool

In [21]:
df_cars["vehicles"]

0        seat
1    mercedes
2         NaN
Name: vehicles, dtype: category
Categories (3, object): ['seat' < 'bmw' < 'mercedes']

# Add new columns saving Nan values `get_dummies_nan` 

In [22]:
df_cars = pd.DataFrame([[2,"mercedes","middleclass"], 
                        [np.NaN,"mercedes","middleclass"],
                        [3,"Audi",np.NaN]],
                        columns= ["members","vehicles","status"])


In [23]:
print(df_cars)

   members  vehicles       status
0      2.0  mercedes  middleclass
1      NaN  mercedes  middleclass
2      3.0      Audi          NaN


In [24]:

def contains_nan(df_col):
    '''
    `contains_nan` checks if a certain column has nans
    '''
    return df_col.isna().any()


def get_dummies_nan(df, return_nancols=False,inplace=False):
    '''
    `get_dummies_nan` creates a new dataframe with binary columns stating wheather variables contain NaNs.
    
    
    
    Examples:
    --------
    
    
    >>> df = pd.DataFrame([[2,"mercedes","middleclass"], 
                           [np.NaN,"mercedes","middleclass"],
                           [3,"Audi",np.NaN]],
                           columns= ["members","vehicles","status"])
    
    >>> df 

           members  vehicles       status
    0      2.0  mercedes  middleclass
    1      NaN  mercedes  middleclass
    2      3.0      Audi          NaN
    
    >>> df_ = get_dummies_nan(df)
    
    >>> df_
    
           members  vehicles       status  members_nan  status_nan
    0      2.0  mercedes  middleclass        False       False
    1      NaN  mercedes  middleclass         True       False
    2      3.0      Audi          NaN        False        True

    >>> df_, nancols = get_dummies_nan(df, return_nancols=True)
    
    >>> nancols
    
    ['members', 'status']

    '''
    def add_nan_columns(df):
        cols_with_nan = []
        for c in df.columns:
            if contains_nan(df[c]):
                cols_with_nan.append(c)
                df[c + "_nan"] = df[c].isna().values    
        return df, cols_with_nan


    if inplace:
        df, cols_with_nan = add_nan_columns(df) 
        if return_nancols:
            return cols_with_nan
    else:
        df_copy = df.copy(deep=True)
        df_copy, cols_with_nan = add_nan_columns(df_copy) 
        
        if return_nancols:
            return df_copy, cols_with_nan
        else:
            return df_copy

In [25]:
df_= get_dummies_nan(df_cars)

In [26]:
df_

,members,vehicles,status,members_nan,status_nan
0,2.0,mercedes,middleclass,False,False
1,NaN,mercedes,middleclass,True,False
2,3.0,Audi,NaN,False,True


In [27]:
df_, nancols = get_dummies_nan(df_cars, return_nancols=True)

In [28]:
nancols

['members', 'status']

# Compute nan features for rows: `nan_features` 

In [29]:

def _nan_rowfeatures(df, reduce_methods=[np.mean, np.std], distances=False, inplace=False):
    
    """
    `nan_features` generates new features containing for a given row k the "statistic" returned by a reduce
    operation on row k.
    
    t also computes the difference between the most value of the transformed nans per row and the found value.

    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                  [3,["d","w","a"]]],columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2  [p, b, None]
    1      3  [a, c, None]
    2      3     [d, w, a]
    
    >>> df_ = nan_rowfeatures(df)
    
    >>> df_

        members  vehicles       status  mean_nans  std_nans
    0      2.0  mercedes  middleclass   0.000000  0.000000
    1      NaN  mercedes  middleclass   0.333333  0.471405
    2      3.0      Audi          NaN   0.333333  0.471405

    """
    def create_col(df, reducer):
        colname     = reducer.__name__ + '_rownans'
        df[colname] = df_cars.apply(lambda x: reducer(pd.isna(x)), axis=1)
        if distances:
            mode        = stats.mode(df[colname])[0]  
            df[colname + '_l1_to_mode' ] = np.abs(df[colname] - mode)
            df[colname + '_l2_to_mode']  = (df[colname] - mode)**2
            
    if inplace:
        for reducer in reduce_methods:
            create_col(df, reducer)
    else:
        df_copy = df.copy(deep=True)
        
        for reducer in reduce_methods:
            create_col(df_copy, reducer)
        return df_copy
    
    

def nan_rowfeatures(df, reduce_methods=[np.mean, np.std], distances=False, inplace=False):
    
    """
    `nan_features` generates new features containing for a given row k the "statistic" returned by a reduce
    operation on row k.
    
    It also computes the difference between the most value of the transformed nans per row and the found value.
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                  [3,["d","w","a"]]],columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2  [p, b, None]
    1      3  [a, c, None]
    2      3     [d, w, a]
    
    >>> df_ = nan_rowfeatures(df)
    
    >>> df_

       members  vehicles       status  mean_rownans  std_rownans
    0      2.0  mercedes  middleclass      0.000000     0.000000
    1      NaN  mercedes  middleclass      0.333333     0.433013
    2      3.0      Audi          NaN      0.333333     0.433013
    
    >>> df_ = nan_rowfeatures(df, distances=True)

           members  vehicles       status  mean_rownans  mean_rownans_l1_to_mode  \
    0      2.0  mercedes  middleclass      0.000000                 0.333333   
    1      NaN  mercedes  middleclass      0.333333                 0.000000   
    2      3.0      Audi          NaN      0.333333                 0.000000   

       mean_rownans_l2_to_mode  std_rownans  std_rownans_l1_to_mode  \
    0                 0.111111     0.000000                0.372678   
    1                 0.000000     0.372678                0.000000   
    2                 0.000000     0.372678                0.000000   

       std_rownans_l2_to_mode  
    0                0.138889  
    1                0.000000  
    2                0.000000  

    """
    
    def create_col(df, reducer):
        colname                       = reducer.__name__ + '_rownans'
        df[colname]                   = reducer(pd.isna(df), axis=1)
        if distances:
            mode                          = stats.mode(df[colname])[0]  
            df[colname + '_l1_to_mode']   = np.abs(mode - df[colname])
            df[colname + '_l2_to_mode']   = (mode - df[colname])**2
            
    if inplace:
        for reducer in reduce_methods:
            create_col(df, reducer)
    else:
        df_copy = df.copy(deep=True)
        
        for reducer in reduce_methods:
            create_col(df_copy, reducer)

        return df_copy

In [30]:
%%time
_nan_rowfeatures(df_cars)

CPU times: user 6.13 ms, sys: 1.28 ms, total: 7.41 ms
Wall time: 7.38 ms


,members,vehicles,status,mean_rownans,std_rownans
0,2.0,mercedes,middleclass,0.000000,0.000000
1,NaN,mercedes,middleclass,0.333333,0.471405
2,3.0,Audi,NaN,0.333333,0.471405


In [31]:
%%time
nan_rowfeatures(df_cars)

CPU times: user 4.57 ms, sys: 1.3 ms, total: 5.87 ms
Wall time: 6.36 ms


,members,vehicles,status,mean_rownans,std_rownans
0,2.0,mercedes,middleclass,0.000000,0.000000
1,NaN,mercedes,middleclass,0.333333,0.433013
2,3.0,Audi,NaN,0.333333,0.433013


In [32]:
nan_rowfeatures(df_cars, distances=True)

,members,vehicles,status,mean_rownans,mean_rownans_l1_to_mode,mean_rownans_l2_to_mode,std_rownans,std_rownans_l1_to_mode,std_rownans_l2_to_mode
0,2.0,mercedes,middleclass,0.000000,0.333333,0.111111,0.000000,0.372678,0.138889
1,NaN,mercedes,middleclass,0.333333,0.000000,0.000000,0.372678,0.000000,0.000000
2,3.0,Audi,NaN,0.333333,0.000000,0.000000,0.372678,0.000000,0.000000


# Substitute `np.NaN` by other values: `MissingImputer` class

In [33]:
from sklearn.base import BaseEstimator, TransformerMixin

In [34]:

class _MissingImputer(BaseEstimator, TransformerMixin):
    '''
    `MissingImputer` implements a `fit` and `transform` methods that enable replacing np.NaN values by numerical values.

    This class uses an insane amount of RAM.
    '''
    def __init__(self, treatment="mean"):
        self._allowed_treatments = ["fixed_value", "mean",'median','mode','None',"most_frequent"]     
        assert treatment in self._allowed_treatments or isinstance(treatment,(int,float)),  "the treatment introduced {} is not valid. Please use one in {}".format(treatment, self._allowed_treatments)
        self.treatment = treatment
    
    def fit(self, X, y):
        """
        Learns statistics to impute nans.
        """
        
        if self.treatment == "mean" or self.treatment==None:
            self.treatment_method = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='mean')
        elif self.treatment == "median":
            self.treatment_method = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='median')
        elif self.treatment == "most_frequent":
            self.treatment_method = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        elif isinstance(self.treatment, (int,float)):
            self.treatment_method = sklearn.impute.SimpleImputer(missing_values=np.nan,
                                                                 strategy="constant",fill_value=self.treatment)       
        
        if isinstance(X, np.ndarray):   
            self.treatment_method.fit(X)
            
        if isinstance(X, pd.DataFrame):       
            self.treatment_method.fit(X.values)
        
        return self

    def transform(self, X):
        if self.treatment==None:
            return X
        return self.treatment_method.transform(X)



In [35]:
import sklearn
from sklearn import *
d = sklearn.datasets.load_iris()
X, y = d["data"], d["target"]

X[0,0] = np.NaN
X[1,2] = np.NaN
X[2,3] = np.NaN
X[0,2] = np.NaN

In [36]:
X[0:3]

array([[nan, 3.5, nan, 0.2],
       [4.9, 3. , nan, 0.2],
       [4.7, 3.2, 1.3, nan]])

In [37]:
imputer = _MissingImputer(treatment="mean")
imputer.fit(X,y)
imputer.transform(X)[0:3]

array([[5.84832215, 3.5       , 3.78986486, 0.2       ],
       [4.9       , 3.        , 3.78986486, 0.2       ],
       [4.7       , 3.2       , 1.3       , 1.20604027]])

In [38]:
imputer = _MissingImputer(treatment="median")
imputer.fit(X,y)
imputer.transform(X)[0:3]

array([[5.8, 3.5, 4.4, 0.2],
       [4.9, 3. , 4.4, 0.2],
       [4.7, 3.2, 1.3, 1.3]])

In [39]:
imputer = _MissingImputer(treatment="most_frequent")
imputer.fit(X,y)
imputer.transform(X)[0:3]

array([[5. , 3.5, 1.5, 0.2],
       [4.9, 3. , 1.5, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [40]:
%%timeit
imputer = _MissingImputer(treatment="most_frequent")
imputer.fit(X,y)

1.04 ms ± 352 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Improving MissingImputer

make it work for dataframes and numpy arrays in a natural way

In [41]:

class NaNTransformer():
    def __init__(self, reducer=stats.mode):
        self.reducer = reducer
        self.col_id  = None 
        
    def fit(self, X_col, col_id):
        self.col_id = col_id
        self.learned_value = self.reducer(X_col)[0]
        
    def transform(self, X_col):
        X_col[pd.isna(X_col)] = self.learned_value
        return X_col

def itercols(X):
    
    if isinstance(X, np.ndarray):
        for i in range(X.shape[1]):
            yield i, X[:,i]
            
    if isinstance(X, pd.DataFrame):
        for colname in X:
            yield colname, X[colname]  
        
def get_col(X, col_id):
    
    if isinstance(X, np.ndarray):
        return  X[:,col_id]
            
    if isinstance(X, pd.DataFrame):
        return  X[col_id]  
             
        
class MissingImputer(BaseEstimator, TransformerMixin):
    '''
    `MissingImputer` implements a `fit` and `transform` methods that enable replacing np.NaN values by numerical values.

    This class less RAM han _MissingImputer
    '''
    def __init__(self, treatment=stats.mode):
        self._allowed_treatments = [np.mean, np.median, stats.mode]     
        assert treatment in self._allowed_treatments or isinstance(treatment,(int,float)),  "the treatment introduced {} is not valid. Please use one in {}".format(treatment, self._allowed_treatments)
        self.treatment = treatment
    
    def fit(self, X, y):
        """
        Learns statistics to impute nans.
        """
        
        col_transformers = {}
        
        for col_id, X_col in itercols(X):
            nan_transformer = NaNTransformer(reducer=self.treatment) 
            nan_transformer.fit(X_col, col_id = col_id)
            col_transformers[col_id] = nan_transformer            
                
        self.col_transformers = col_transformers
        return self

    def transform(self, X, inplace = False):
        """
        Method for transforming a dataframe of numpy array X replacing the np.NaN values.
        
        Notice that this method has a for loop over `self.col_transformers` instead of iterating over `itercols(X)`
        the main reason is that after `fit(X,y)` is done, a user might change `X`, introducing new columns that we
        don't want to iterave over (because we don't have a col_transformer for those new columns).

        -  If X is a dataframe users can change the order of the columns and still be able to use the transform method.
        
        -  If X is a numpy array then results will be corrupted because we don't have pointers to columns.
           Notice that if X is a numpy array `col_transformer` uses the indices of the columns found during `.fit(X,y)`.

        """
        
        if inplace:

            if self.treatment==None:
                return None

            #for col_id, X_col in itercols(X):
            #    X_col = self.col_transformers[col_id].transform(X_col)
            for col_id in self.nan_transformers:
                X_col = get_col(X, col_id)
                X_col = self.col_transformers[col_id].transform(X_col)
                
        else:
            X_copy = copy.deepcopy(X)
            
            if self.treatment==None:
                return X_copy
            
            #for col_id, X_col in itercols(X_copy):
            #    X_col = self.col_transformers[col_id].transform(X_col)
            for col_id in self.col_transformers:
                X_col = get_col(X_copy, col_id)
                X_col = self.col_transformers[col_id].transform(X_col)
                
            return X_copy

In [42]:
import sklearn
from sklearn import *
d = sklearn.datasets.load_iris()
X, y = d["data"], d["target"]

X[0,0] = np.NaN
X[1,2] = np.NaN
X[2,3] = np.NaN
X[0,2] = np.NaN

In [43]:
X[0:4]

array([[nan, 3.5, nan, 0.2],
       [4.9, 3. , nan, 0.2],
       [4.7, 3.2, 1.3, nan],
       [4.6, 3.1, 1.5, 0.2]])

In [44]:
pd.isna(X[0:4])

array([[ True, False,  True, False],
       [False, False,  True, False],
       [False, False, False,  True],
       [False, False, False, False]])

In [45]:
imputer = MissingImputer()

In [46]:
imputer.fit(X,y)

MissingImputer(treatment=<function mode at 0x1521dad048>)

In [47]:
imputer.col_transformers

{0: <__main__.NaNTransformer at 0x1a235d0278>,
 1: <__main__.NaNTransformer at 0x1a22626438>,
 2: <__main__.NaNTransformer at 0x1a235d02e8>,
 3: <__main__.NaNTransformer at 0x1a235b7f28>}

In [48]:
#%%timeit
imputer = MissingImputer(treatment=stats.mode)
imputer.fit(X,y)

a = imputer.transform(X)

In [49]:
a[0:4]

array([[5. , 3.5, 1.5, 0.2],
       [4.9, 3. , 1.5, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2]])

In [50]:
X[0:4]

array([[nan, 3.5, nan, 0.2],
       [4.9, 3. , nan, 0.2],
       [4.7, 3.2, 1.3, nan],
       [4.6, 3.1, 1.5, 0.2]])

# New columns from a df column containing lists as items (`byvalue`)

In [51]:
df_cars = pd.DataFrame([[2,["mercedes","toyota",np.nan],"middleclass"], 
                        [3,["Renault","Mercedes",np.nan],"middleclass"],
                        [3,["Audi","Mercedes","Tesla"],"uppermiddleclass"]],
                        columns= ["members","vehicles","status"])

df_cars

,members,vehicles,status
0,2,"[mercedes, toyota, nan]",middleclass
1,3,"[Renault, Mercedes, nan]",middleclass
2,3,"[Audi, Mercedes, Tesla]",uppermiddleclass


In [52]:
def onehot_collist_byvalue(collist, values_to_index):
    """
    
    `onehot_collist_byvalue` generates a np.ndarray containing the one hot representation of the input collist
     The returned array stores at position `(i,j)` a 1 if the linked column to integer `j` has a name  found 
     in `collist[i]`.

    ##### Example 
    
    collist         = [["tesla"], ["mercedes", "audi"], ["toyota", "mercedes", "tesla"]]
    values_to_index = {'audi':0, 'mercedes':1, 'tesla':2. 'toyota':3}
    
    The returned array should be:

        [0 0 1 0
         1 1 0 0
         0 1 1 1]
    
    ## INPUTS
    
    values_to_index: dict
    
    Dictionary asigning an index to all the possible values in the different lists o collist.
    
    
    ## OUPTUT
    
    onehot_matrix: np.ndarray
    
    Numpy matrix storing at position `(i,j)` the existance of value `j` in the list `collist[i]`.
    
    """
    
    n_rows = len(collist)
    n_cols = len(values_to_index)
    result = np.zeros((n_rows, n_cols), dtype=np.bool)   
    
    for k,row in enumerate(collist):
        for element in row:
            element_index = values_to_index[str(element)]
            result[k, element_index] = 1
            
    return result
            

In [53]:
collist = [["tesla"], ["mercedes", "audi"], ["toyota", "mercedes", "tesla"], ["mercedes"], ["audi"]]

In [54]:
colname = "vehicles"
unique_values = list(set([str(x).lower() for x in itertools.chain(*df_cars[colname].values)]))
colnames   = [colname + "_" + str(i) for i in unique_values]   
n_unique_values = len(unique_values)
colnames.sort()
unique_values.sort()
unique_values_to_index = {val:i for i,val in enumerate(unique_values)} 

In [55]:
unique_values_to_index

{'audi': 0, 'mercedes': 1, 'nan': 2, 'renault': 3, 'tesla': 4, 'toyota': 5}

In [56]:
onehot_collist_byvalue(collist, unique_values_to_index)

array([[False, False, False, False,  True, False],
       [ True,  True, False, False, False, False],
       [False,  True, False, False,  True,  True],
       [False,  True, False, False, False, False],
       [ True, False, False, False, False, False]])

Now we can call `onehot_collist_byvalue` inside `proc_df_collist_byvalue` and return a pandas dataframe

In [57]:
def proc_df_collist_byvalue(df: pd.DataFrame, colname: str, inplace=False):
    """
    
    `proc_df_collist` takes a dataframe and a column name as input. If the column of the dataframe
    is made of iterables then it generates new columns. Otherwise it returns an assertion error.
    
    The function generates new columns, as many as different values can be found in the elements of the iterables
    in `df[colname]`. Each new column stores, at position `k`, the presence (or absence) of an item in `df[colname][k]`.

    - Each new column is formated as "colname_X" where X is one of the values found in `pd[colname]`.
    
    - The function returns a new dataframe onehot_df if inplace=False.
    
    - `onehot_df[colname_X][k]` is 1 if and only if `X in pd[colname][k]` (it's zero otherwise).
    
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",np.nan]], 
                           [3,["a","c",np.nan]],
                           [3,["d","w","a"]]],
                           columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2   [p, b, nan]
    1      3   [a, c, nan]
    2      3     [d, w, a]

    >>> newcols = proc_df_collist_byorder(df, "second")

    >>> newcols
    
       second_a  second_b  second_c  second_d  second_nan  second_p  second_w
    0     False      True     False     False        True      True     False
    1      True     False      True     False        True     False     False
    2      True     False     False      True       False     False      True

    
    >>> df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])
                   
    >>> df2
    
       A          B
    0  2        [p]
    1  3  [a, 2, 3]
    2  3        [4]

    >>> proc_df_collist_byorder(df2, "B")

      B_0  B_1  B_2
    0   p  NaN  NaN
    1   a  2.0  3.0
    2   4  NaN  NaN

    """
    assert isinstance(colname, str), "type(columname)={} but it should be str".format(type(str))
    assert isinstance(df[colname].iloc[0],(list,set, np.ndarray)), "type(df[colname].iloc[0])={} but it, should be in [list, set, np.ndarray]".format(type(df[colname].iloc[0]))
    
    unique_values = list(set([str(x) for x in itertools.chain(*df[colname].values)]))
    colnames   = [colname + "_" + str(i) for i in unique_values]   
    n_unique_values = len(unique_values)
    colnames.sort()
    unique_values.sort()
    
    unique_values_to_index = {val:i for i,val in enumerate(unique_values)} 
    onehot_array = onehot_collist_byvalue(df[colname], unique_values_to_index)
    
    onehot_df    = pd.DataFrame(onehot_array, columns=[f"{colname}_{c}" for c in unique_values])
    
    return onehot_df 

In [58]:
df = pd.DataFrame([[2,["p","b",np.nan]], 
                   [3,["a","c",np.nan]],
                   [3,["d","w","a"]]],
                   columns= ["first","second"])

In [59]:
df

,first,second
0,2,"[p, b, nan]"
1,3,"[a, c, nan]"
2,3,"[d, w, a]"


In [60]:
aux = proc_df_collist_byvalue(df, "second")

In [61]:
print(str(aux))

   second_a  second_b  second_c  second_d  second_nan  second_p  second_w
0     False      True     False     False        True      True     False
1      True     False      True     False        True     False     False
2      True     False     False      True       False     False      True


In [62]:
aux

,second_a,second_b,second_c,second_d,second_nan,second_p,second_w
0,False,True,False,False,True,True,False
1,True,False,True,False,True,False,False
2,True,False,False,True,False,False,True


In [63]:
def proc_df_collist_byvalue(df: pd.DataFrame, colname: str, inplace=False):
    """
    
    `proc_df_collist` takes a dataframe and a column name as input. If the column of the dataframe
    is made of iterables then it generates new columns. Otherwise it returns an assertion error.
    
    The function generates new columns, as many as different values can be found in the elements of the iterables
    in `df[colname]`. Each new column stores, at position `k`, the presence (or absence) of an item in `df[colname][k]`.

    - Each new column is formated as "colname_X" where X is one of the values found in `pd[colname]`.
    
    - The function returns a new dataframe onehot_df if inplace=False.
    
    - `onehot_df[colname_X][k]` is 1 if and only if `X in pd[colname][k]` (it's zero otherwise).
    
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",np.nan]], 
                           [3,["a","c",np.nan]],
                           [3,["d","w","a"]]],
                           columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2   [p, b, nan]
    1      3   [a, c, nan]
    2      3     [d, w, a]

    >>> newcols = proc_df_collist_byorder(df, "second")

    >>> newcols
    
       second_a  second_b  second_c  second_d  second_nan  second_p  second_w
    0     False      True     False     False        True      True     False
    1      True     False      True     False        True     False     False
    2      True     False     False      True       False     False      True

    
    >>> df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])
                   
    >>> df2
    
       A          B
    0  2        [p]
    1  3  [a, 2, 3]
    2  3        [4]

    >>> proc_df_collist_byorder(df2, "B")

      B_0  B_1  B_2
    0   p  NaN  NaN
    1   a  2.0  3.0
    2   4  NaN  NaN

    """
    assert isinstance(colname, str), "type(columname)={} but it should be str".format(type(str))
    assert isinstance(df[colname].iloc[0],(list,set, np.ndarray)), "type(df[colname].iloc[0])={} but it, should be in [list, set, np.ndarray]".format(type(df[colname].iloc[0]))
    
    unique_values = list(set([str(x) for x in itertools.chain(*df[colname].values)]))
    colnames   = [colname + "_" + str(i) for i in unique_values]   
    n_unique_values = len(unique_values)
    colnames.sort()
    unique_values.sort()
    
    unique_values_to_index = {val:i for i,val in enumerate(unique_values)} 
    onehot_array = onehot_collist_byvalue(df[colname], unique_values_to_index)
    
    onehot_df    = pd.DataFrame(onehot_array, columns=[f"{colname}_{c}" for c in unique_values])
    
    return onehot_df 

## New columns from a df column containing lists as items (`bykeyvalue`)

Let us imagine the scenario that a column contains strings of the form "variable=value" (or maybe "variable_value" or any other way to encode pairs of elements as a string). A column could even contain lists of elements of this form, for example `["var1=v1", "var2=v2"]`. Asuming we have lots of possible values for the different variables we might want to avoind encoding our data in a onehot representation.  Therefore, we want an alternative to `proc_df_collist_byvalue` which writes a column for every possible value of `colname`.

Then we can pass a function `get_key_and_value` that should return the part of the string we are interested to use as a new column (the key) as well as the part of the string we want to use as value (the value)

For example if we have
```
["v1=23", "v2=234", "v1=5", "v2=7"]
```

We might want to generate 2 columns, one for "v1" and one for "v2", containing the values (23, 0, 5, 0) and (0,234,0,7) respectively.

In [64]:
df = pd.DataFrame([[2, ["var1=p","var2=p",np.nan]], 
                   [3, ["var2=p","var1=pc",np.nan]],
                   [3, ["var3=pd","var2=pw","var1=pa"]]],
                   columns= ["first","second"])

In [65]:
df

,first,second
0,2,"[var1=p, var2=p, nan]"
1,3,"[var2=p, var1=pc, nan]"
2,3,"[var3=pd, var2=pw, var1=pa]"


In [66]:
colname = "second"

df[colname]

0          [var1=p, var2=p, nan]
1         [var2=p, var1=pc, nan]
2    [var3=pd, var2=pw, var1=pa]
Name: second, dtype: object

First we can convert values inside the list as strings, moreover we also add the variable "nan" 
to capture np.nan values in a new column (that we will construct later).

In [67]:
def treat_nan(x):
    result = []
    for x_k in x:
        if isinstance(x_k, str):
            result.append(x_k)
        elif np.isnan(x_k):
            result.append("nan=np.nan")
    return result

df[colname] = df[colname].apply(treat_nan)


In [68]:
df[colname]

0     [var1=p, var2=p, nan=np.nan]
1    [var2=p, var1=pc, nan=np.nan]
2      [var3=pd, var2=pw, var1=pa]
Name: second, dtype: object

Now we want to know how many columns we need to create.

A column will be created for every variable (or key). Now we will iterate over the items to extract all keys.
This is done in `get_all_keys`.

In [69]:
def get_key_value(k):
    return k.split("=")

In [70]:
def get_all_keys(df, colname):
    all_keys = set()
    for x in df[colname]:
        all_keys = all_keys.union([get_key_value(x_k)[0] for x_k in x])
    
    return all_keys    

In [71]:
all_keys = list(get_all_keys(df,colname))
all_keys.sort()
all_keys

['nan', 'var1', 'var2', 'var3']

Each value in `all_keys` will generate a new column.
Let us create names for the new columns


In [72]:
newcolnames = [colname + "_" + name for name in all_keys]
newcolnames

['second_nan', 'second_var1', 'second_var2', 'second_var3']

When we create the lists containing the items we need to know at which positions an item has to be written.
We store the variables in a mapping `newcolnames_to_int` that given a column name it returns the position.

In [73]:
newcolnames_to_int = {col:i for i,col in  enumerate(newcolnames)}
newcolnames_to_int

{'second_nan': 0, 'second_var1': 1, 'second_var2': 2, 'second_var3': 3}

We also do the same for the different variables

In [74]:
var_to_pos = {col:newcolnames_to_int[fullname] for col,fullname in  zip(all_keys,newcolnames)}
print(var_to_pos)
pos_to_var  = {v: k for k, v in var_to_pos.items()}
print(pos_to_var)

{'nan': 0, 'var1': 1, 'var2': 2, 'var3': 3}
{0: 'nan', 1: 'var1', 2: 'var2', 3: 'var3'}


Now we want to convert the lists in the selected row (selected by `colname`) to new lists of a fixed size order
containing the original feature values in a sensible representation.

This new representation will have at position `k` the values of the variable `pos_to_var[k]`.

Once this representation is complete for all positions in `df[colname]` then we can simply create a dataframe
of of our list of lists. Since the "sublists" will have all the same length there won't be any problem in 
generating the dataframe. Since the construction process took care of the positions at which items were written
usin the mapping `var_to_pos` we can simply use, as columns of our new dataframe, the names in `newcolnames`.
Notice `var_to_pos` was created using `newcolnames` and we have a bijection between positions of the sublists
and names in `newcolnames`.

In [75]:
def write_row_as_list(row, var_to_pos):
    row_code = [0]*len(var_to_pos)
    
    for item in row:
        key, value = get_key_value(item)
        row_code[var_to_pos[key]] = value
        
    return row_code

In [76]:
row = df["second"].iloc[0]

In [77]:
row

['var1=p', 'var2=p', 'nan=np.nan']

In [78]:
var_to_pos

{'nan': 0, 'var1': 1, 'var2': 2, 'var3': 3}

In [79]:
write_row_as_list(row, var_to_pos)

['np.nan', 'p', 'p', 0]

In [80]:
def get_key_value(k):
    return k.split("=")

def proc_df_collist_bykeyvalue(df: pd.DataFrame, colname: str, get_key_value,
                               column_separator="_", inplace=False, treat_nan_func=None):
    """
    
    `proc_df_collist` takes a dataframe and a column name as input. If the column of the dataframe
    is made of iterables then it generates new columns. Otherwise it returns an assertion error.
    
    The function generates new columns, as many as different "keys"  can be found in the elements of the iterables
    in `df[colname]`. In order to found a "key" from an element `proc_df_collist_bykeyvalue` expects a method
    `get_key_value`, that should return a (key,value) pair for a given input. 
    
    
    Each new column stores, the elements of the correspondant variable.
    
    - Each new column is formated as `f"colname{column_separator}X"` where X is one of the variables found
      in `pd[colname]`.
    
    - The function returns a new dataframe `result_df` if inplace=False.
    
    - `result_df[colname{column_separator}X][k]` has a value if and only if `X in pd[colname][k]` (it's zero otherwise).
    
    
    Example of `get_key_value` method
    
    Assume rows of `df[colname]` have the form [`var1=v1`, `var2=v2`]. Then a possible function `get_key_value`
    could be simply `lambda x: x.split("=")` which returns a pair of elements for a given input string.
            
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["var1=p", "var2=p", np.nan]], 
                           [3,["var2=p", "var1=pc", np.nan]],
                           [3,["var3=pd", "var2=pw", "var1=pa"]]],
                           columns= ["first","second"])
                  
    >>> df

       first                         second
    0      2   [var1=p, var2=p, nan=np.nan]
    1      3  [var2=p, var1=pc, nan=np.nan]
    2      3    [var3=pd, var2=pw, var1=pa]

    >>> colname = "second"
    
    >>> def get_key_value(k):
           return k.split("=")

    >>> proc_df_collist_bykeyvalue(df, colname, get_key_value, column_separator="__")

      second__nan second__var1 second__var2 second__var3
    0      np.nan            p            p            0
    1      np.nan           pc            p            0
    2           0           pa           pw           pd
    
    """
    
    def treat_nan(x):
        result = []
        for x_k in x:
            if isinstance(x_k, str):
                result.append(x_k)
            elif np.isnan(x_k):
                result.append("nan=np.nan")
        return result

    def get_all_keys(df, colname):
        all_keys = set()
        for x in df[colname]:
            all_keys = all_keys.union([get_key_value(x_k)[0] for x_k in x])
        return all_keys   

    def write_row_as_list(row, var_to_pos):
        row_code = [0]*len(var_to_pos)

        for item in row:
            key, value = get_key_value(item)
            row_code[var_to_pos[key]] = value
        return row_code

    if treat_nan_func is None:
        treat_nan = treat_nan
    else:
        treat_nan = treat_nan_func
    
    
    assert isinstance(colname, str), "type(columname)={} but it should be str".format(type(str))
    assert isinstance(df[colname].iloc[0],(list,set, np.ndarray)), "type(df[colname].iloc[0])={} but it, should be in [list, set, np.ndarray]".format(type(df[colname].iloc[0]))

    df[colname] = df[colname].apply(treat_nan)
    all_keys    = list(get_all_keys(df, colname))
    all_keys.sort()
    newcolnames        = [colname + column_separator + name for name in all_keys]
    newcolnames_to_int = {col:i for i,col in  enumerate(newcolnames)}
    var_to_pos         = {col:newcolnames_to_int[fullname] for col,fullname in  zip(all_keys,newcolnames)}
    pos_to_var         = {v: k for k, v in var_to_pos.items()}
    
    result = []
    for row in df[colname]:
        result.append(write_row_as_list(row, var_to_pos))
            
    
    result_df   = pd.DataFrame(result, columns=newcolnames)
    
    return result_df 


In [81]:
df

,first,second
0,2,"[var1=p, var2=p, nan=np.nan]"
1,3,"[var2=p, var1=pc, nan=np.nan]"
2,3,"[var3=pd, var2=pw, var1=pa]"


In [82]:
colname = "second"
def get_key_value(k):
    return k.split("=")

result = proc_df_collist_bykeyvalue(df, colname, get_key_value, column_separator="__")

In [83]:
result

,second__nan,second__var1,second__var2,second__var3
0,np.nan,p,p,0
1,np.nan,pc,p,0
2,0,pa,pw,pd


# New columns from a df column containing lists as items (`byorder`)

Column `vehicles` contains lists with different vehicle names. Let us assume we consider this feature to be a list or the ordered vehicles a family has.

For example: Family 0 has 2 vehicles, and the most used one is a Mercedes, then a Toyota.

Now we want to create 3 features from this column: `vehicle_1`, `vehicle_2`, `vehicle_3` and write the different
items in the corresponding columns (the corresponding column depends on the position of the elements in the list, not on the values of the elements in the list). If you want to create `vehicle_mercedes`, `vehicle_toyota` etc look at the previous function `proc_df_collist_byvalue`.

In [84]:
df_cars = pd.DataFrame([[2,["mercedes","toyota",np.nan],"middleclass"], 
                        [3,["Renault","Mercedes",np.nan],"middleclass"],
                        [3,["Audi","Mercedes","Tesla"],"uppermiddleclass"]],
                        columns= ["members","vehicles","status"])

df_cars

,members,vehicles,status
0,2,"[mercedes, toyota, nan]",middleclass
1,3,"[Renault, Mercedes, nan]",middleclass
2,3,"[Audi, Mercedes, Tesla]",uppermiddleclass


In [85]:
def proc_df_collist_byorder(df: pd.DataFrame, colname: str, inplace=False):
    """
    
    `proc_df_collist` takes a dataframe and a column made of lists and generates new columns which write the
    values contained in the original lists in a consecutive order.
    
    For each position in the list it generates a new column. The total number of columns 
    equals the length of the largest list in `df[colname]`. 
    
    - Each new collumn  k is filled with the values of the lists at position k.
    
    - If the value does not exist (because the position does not exist) the position is filled with `NaN`. 
    
    Given `df`  and `colname`, create as many new columns as `n_new_cols =  df[colname].apply(len).max()`
    Write in column `colname_k[j]` the value found `df[colname].iloc[j][k]`. If a position for a given row
    does not exist, write np.nan in the corresponding column.
    
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2, ["p", "b", np.nan]], 
                           [3, ["a", "c", np.nan]],
                           [3, ["d", "w", "a"]]],
                           columns= ["first","second"])

    >>> df
    
       first        second
    0      2  [p, b, nan]
    1      3  [a, c, nan]
    2      3     [d, w, a]

    >>> newcols = proc_df_collist_byorder(df, "second")

    >>> newcols
    
          second_0 second_1 second_2
    0        p        b     None
    1        a        c     None
    2        d        w        a

    
    >>> df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])
                   
    >>> df2
    
       A          B
    0  2        [p]
    1  3  [a, 2, 3]
    2  3        [4]

    >>> proc_df_collist_byorder(df2, "B")

      B_0  B_1  B_2
    0   p  NaN  NaN
    1   a  2.0  3.0
    2   4  NaN  NaN

    """
    assert isinstance(df, pd.DataFrame), "type(df)={} but it should be pd.DataFrame".format(type(df))
    assert isinstance(colname, str), "type(columname)={} but it should be str".format(type(str))
    assert isinstance(df[colname].iloc[0],(list,set, np.ndarray)), "type(df[colname].iloc[0])={} but it, should be in [list, set, np.ndarray]".format(type(df[colname].iloc[0]))
    
    #n_new_cols = len(df[colname].iloc[0])
    n_new_cols = df[colname].apply(len).max()
    colnames   = [colname + "_" + str(i) for i in range(n_new_cols)]   
    return pd.DataFrame(df[colname].tolist(), columns=colnames)

In [86]:
df = pd.DataFrame([[2, ["p", "b", np.nan]], 
                   [3, ["a", "c", np.nan]],
                   [3, ["d", "w", "a"]]],
                   columns= ["first", "second"])

In [87]:
df

,first,second
0,2,"[p, b, nan]"
1,3,"[a, c, nan]"
2,3,"[d, w, a]"


In [88]:
proc_df_collist_byorder(df, "second")

,second_0,second_1,second_2
0,p,b,NaN
1,a,c,NaN
2,d,w,a


If the column passed to `proc_df_collist` contains iterables with different sizes then it will generate as many colums as the longest iterable. Then it will fill with `NaN`  positions where we don't have information

In [89]:
df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])

In [90]:
proc_df_collist_byorder(df2, "B")

,B_0,B_1,B_2
0,p,NaN,NaN
1,a,2.0,3.0
2,4,NaN,NaN
